In this notebook we will work through a first-level model using the GLM tools within nipy.  These are not as widely used as the FSL tools, but they run much faster and afford us some more flexibility.

In [ ]:
%pylab inline
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import nibabel as nib
print "nibabel:", nib.__version__
import nipy
print "nipy:", nipy.__version__

In [ ]:
img = nib.load('srabold.nii.gz')

In [ ]:
print(img.header)

Defaults for plotting

In [ ]:
plt.rcParams['image.cmap'] = 'gray'
plt.rcParams['image.interpolation'] = 'nearest'

Loading the image may take a little while:

In [ ]:
data = img.get_data()
data.shape

In [ ]:
plt.imshow(data.mean(axis=-1)[:, :, 30])

Why is this image blurry?

So what was the subject doing here?

https://openfmri.org/dataset/ds000105

* 001 House
* 002 Scrambled
* 003 Cat
* 004 Shoe
* 005 Bottle
* 006 Scissors
* 007 Chair
* 008 Face

In [ ]:
event_types = ['house', 'scrambled', 'cat', 'shoe', 'bottle', 'scissors', 'chair', 'face']

You should have the task lists for this subject in your `~/data/ds105` directory.  Let's check:

In [ ]:
import os
from os import path as osp
HOME = os.path.expanduser('~')
DATA_PATH = os.path.join(HOME, 'data', 'ds105')
SUB1_TASK_PATH = os.path.join(DATA_PATH, 'sub001', 'model', 'model001', 'onsets', 'task001_run001')
os.path.isdir(SUB1_TASK_PATH)

If you got "False" above - you can download the zip file of the task files you need from:

http://nipy.bic.berkeley.edu/practical_neuroimaging/ds105_sub001_tasks.zip

Download the file, then `cd ~/data` and `unzip ~/Downloads/ds105_sub001_tasks.zip` (if you downloaded the file to your `Downloads` folder.

Recheck:

In [ ]:
os.path.isdir(SUB1_TASK_PATH)

Look in this path:

In [ ]:
os.listdir(SUB1_TASK_PATH)

What are in these files?

In [ ]:
COND1_FNAME = os.path.join(SUB1_TASK_PATH, 'cond001.txt')
print(open(COND1_FNAME, 'rt').read())

This is a common format for specifying events - any guesses what the columns are?


We can actually do :

In [ ]:
dtype_def = [('onset', 'f'), ('duration', 'f'), ('amplitude', 'f')]
evs = np.loadtxt(COND1_FNAME, dtype_def)
evs

This is nice because:

In [ ]:
evs[0]['onset']

In [ ]:
evs['onset']

We can load our events for all types:

In [ ]:
# create event_defs : a dictionary with keys the type of events, ('bottle', 'cat', ..)
event_defs = {}
for eno in range(8):
    fname = os.path.join(os.path.join(SUB1_TASK_PATH, 'cond%03d.txt' % (eno + 1)))
    print(fname)
    event_name = event_types[eno]
    event_defs[event_name] = np.loadtxt(fname, dtype_def)
event_defs          

Are these onsets in TRs or in seconds?

OK - how to make our design?

In [ ]:
# use to be : import nipy.modalities.fmri.design as fmrid #
from nipy.modalities.fmri.experimental_paradigm import EventRelatedParadigm
import nipy.modalities.fmri.design_matrix as dm

#from glob import glob

EventRelatedParadigm takes a list of string and events

In [ ]:
n_scans = img.shape[-1]
TR = 2.5
t = np.arange(n_trs) * TR

In [ ]:
print(event_defs.keys())
print(event_defs['house']['onset'])

In [ ]:
conditions = []
onsets = []

for cond in event_defs.keys():
    cond_onsets = event_defs[cond]['onset']
    conditions += [cond]*len(cond_onsets)
    onsets += list(cond_onsets)

print(conditions)
print(onsets)

In [ ]:
# make paradigm with conditions and onsets (you could add durations and magnitude here)
paradigm = EventRelatedParadigm(conditions, onsets)

hrf_model = 'canonical'
drift_model = 'cosine'
# here you would pick the motion regressors
motion_reg = np.cumsum(np.random.randn(n_scans, 6), 0)
motion_reg_names = ['tx', 'ty', 'tz', 'rx', 'ry', 'rz']
frametimes = np.linspace(0, (n_scans - 1) * TR, n_scans)

Nice - so let's make the design:

In [ ]:
X, names = dm.dmtx_light(frametimes, paradigm, drift_model='cosine',
                         hfcut=128, hrf_model=hrf_model, add_regs=motion_reg,
                         add_reg_names=motion_reg_names)


In [ ]:
plt.imshow(X/X.std(axis=0), interpolation='nearest', aspect='auto')

In [ ]:
from nipy.modalities.fmri.glm import GeneralLinearModel


sh = img.shape[:-1]
Y = img.get_data().reshape(np.prod(sh), n_scans)
glm = GeneralLinearModel(X)
glm.fit(Y.T)


In [ ]:
# specify the contrast [1 -1 0 ..]
contrast = np.zeros(X.shape[1])
contrast[:8] = [-1, -1, -1, -1, -1, -1, -1, 7]


In [ ]:
# compute the constrast image related to it
zvals = glm.contrast(contrast).z_score()
contrast_image = nib.Nifti1Image(np.reshape(zvals, sh), img.get_affine())


In [ ]:
con_img = np.reshape(contrast_image.get_data(), img.shape[:-1])
con_img.shape

In [ ]:
plt.imshow(con_img[..., 30])

Can you find any face activity you believe?